In [20]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('ASEP_questions_data.csv', header=None)

# Preview the data
df

,0
0,1. Η ελευθερία έκφρασης γνώμης ασκείται:
1,α. Από τους/τις Έλληνες/ίδες πολίτες
2,β. Από όσους/ες ευρίσκονται στην Ελληνική επικ...
3,γ. Από τους/τις Έλληνες/ίδες πολίτες και τους/...
4,δ. Από τους/τις κατόχους της κοινής Ευρωπαϊκής...
...,...
2670,41. Οι Ολυμπιακοί Αγώνες της σύγχρονης εποχής ...
2671,α. Μία φορά το 2004
2672,β. Δύο φορές το 1896 και το 2004
2673,"γ. Τρεις φορές το 1896, το 1902 και το 2004"


In [21]:
df.head()

,0
0,1. Η ελευθερία έκφρασης γνώμης ασκείται:
1,α. Από τους/τις Έλληνες/ίδες πολίτες
2,β. Από όσους/ες ευρίσκονται στην Ελληνική επικ...
3,γ. Από τους/τις Έλληνες/ίδες πολίτες και τους/...
4,δ. Από τους/τις κατόχους της κοινής Ευρωπαϊκής...


In [23]:
# Keep only rows that don't contain the phrase
df_cleaned = df[~df.astype(str).apply(lambda row: row.str.contains("Παρασκευή 20 Ιουνίου 2025 έως και Τρίτη 1 Ιουλίου 2025").any(), axis=1)]
df_cleaned.shape

(2607, 1)

In [24]:
import re

# Replace all substrings like "Σελίδα 10 από 69", "Σελίδα 1 από 69", etc. with empty string
df_cleaned_without_selida = df_cleaned.replace(r"Σελίδα\s+\d+\s+από\s+69", "", regex=True)
df_cleaned_without_selida.shape


(2607, 1)

In [25]:
df_cleaned_without_selida

,0
0,1. Η ελευθερία έκφρασης γνώμης ασκείται:
1,α. Από τους/τις Έλληνες/ίδες πολίτες
2,β. Από όσους/ες ευρίσκονται στην Ελληνική επικ...
3,γ. Από τους/τις Έλληνες/ίδες πολίτες και τους/...
4,δ. Από τους/τις κατόχους της κοινής Ευρωπαϊκής...
...,...
2670,41. Οι Ολυμπιακοί Αγώνες της σύγχρονης εποχής ...
2671,α. Μία φορά το 2004
2672,β. Δύο φορές το 1896 και το 2004
2673,"γ. Τρεις φορές το 1896, το 1902 και το 2004"


In [26]:
# Combine all cells from the relevant column into a list
lines = df_cleaned_without_selida.iloc[:, 0].fillna("").tolist()

# Function to check if a line starts with a number+dot or α./β./γ./δ.
def is_starter(line):
    return bool(re.match(r"^\d+\.\s|^[αβγδ]\.", line.strip()))

# Output list
cleaned = []

for line in lines:
    line = line.strip()
    if not line:
        continue  # skip blank lines
    if is_starter(line):
        cleaned.append(line)
    else:
        # Append to last item if available
        if cleaned:
            cleaned[-1] += " " + line
        else:
            cleaned.append(line)  # fallback for first row being non-starter

# Save or display result
result_df = pd.DataFrame(cleaned, columns=["Cleaned"])

In [27]:
result_df.shape

(1989, 1)

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [38]:
# Let's assume your column is named 'text'
lines = result_df['Cleaned'].dropna().astype(str).tolist()

questions = []
current_question = {}
current_choices = {'α': '', 'β': '', 'γ': '', 'δ': ''}

for line in lines:
    line = line.strip()

    # Start of a new question (e.g., "40. Ποιο...")
    if re.match(r'^\d+\.', line):
        if current_question:
            current_question.update(current_choices)
            questions.append(current_question)
            current_choices = {'α': '', 'β': '', 'γ': '', 'δ': ''}
        qnum, qtext = line.split('.', 1)
        current_question = {'Question #': qnum.strip(), 'Question': qtext.strip()}

    # Start of a new answer (e.g., "α. ...")
    elif re.match(r'^[α-δ]\.', line):
        option = line[0]
        current_choices[option] = line[2:].strip()

    # Continuation of a previous answer (no new marker)
    elif any(current_choices[opt] for opt in current_choices):
        for opt in current_choices:
            if current_choices[opt] and not current_choices[opt].endswith('.'):
                current_choices[opt] += ' ' + line
                break

# Add the last question
if current_question:
    current_question.update(current_choices)
    questions.append(current_question)

# Convert to DataFrame
clean_df = pd.DataFrame(questions)

In [39]:
clean_df

,Question #,Question,α,β,γ,δ
0,1,Η ελευθερία έκφρασης γνώμης ασκείται:,Από τους/τις Έλληνες/ίδες πολίτες,Από όσους/ες ευρίσκονται στην Ελληνική επικράτεια,Από τους/τις Έλληνες/ίδες πολίτες και τους/τις Έλληνες/ίδες ομογενείς,Από τους/τις κατόχους της κοινής Ευρωπαϊκής ιθαγένειας
1,2,Τα βασανιστήρια:,Επιτρέπονται για την αποτροπή τρομοκρατικής επίθεσης,Δεν επιτρέπονται σε καμία περίπτωση,Επιτρέπονται με νόμο που ψηφίζει η Βουλή με τα 3/5 των μελών της,Επιτρέπονται για την προστασία της εδαφικής ακεραιότητας της χώρας
2,3,Το δικαίωμα επιλογής επαγγέλματος:,Αποτελεί έκφραση της ελευθερίας συμμετοχής στην κοινωνική ζωή της Χώρας,Δεν αποτελεί έκφραση της ελευθερίας συμμετοχής στην οικονομική ζωή της Χώρας,Δεν αποτελεί δικαίωμα,Είναι ειδική μορφή της οικονομικής ελευθερίας
3,4,Η ελευθερία ανάπτυξης της προσωπικότητας (άρ. 5 παρ. 1 Συντ.) ασκείται:,Μόνο από τους/τις Έλληνες/ίδες πολίτες,"Από τους/τις Έλληνες/ίδες πολίτες, τους/τις Ευρωπαίους/ες πολίτες, τους/τις αλλοδαπούς/ές και τους/τις ανιθαγενείς",Μόνο από τους/τις Έλληνες/ίδες και Ευρωπαίους/ες πολίτες,Μόνο από τους/τις Έλληνες/ίδες πολίτες και τους/τις ομογενείς
4,5,Κατοικία κατά την έννοια του άρ. 9 Συντ. αποτελεί:,Κάθε χώρος που ο άνθρωπος ορίζει ως χώρο διαβιώσεως και εργασίας,Κάθε χώρος που ο άνθρωπος ορίζει ως χώρο διαβιώσεως μόνο,Κάθε χώρος που ο άνθρωπος ορίζει ως χώρο εργασίας μόνο,Κάθε χώρος που ο άνθρωπος διαμένει έστω και για λίγες ώρες όπως το δωμάτιο ξενοδοχείου
5,6,Η ευθανασία:,Είναι συλλογικό δικαίωμα,Είναι κοινωνικό δικαίωμα,Δεν αναγνωρίζεται στο ελληνικό δίκαιο,Κατοχυρώνεται στο Σύνταγμα
6,7,Η αξία του ανθρώπου είναι:,Υποκειμενικό συνταγματικό δικαίωμα,Αντικειμενική συνταγματική αρχή,Νομοθετικός κανόνας,Πολιτικό δικαίωμα
7,8,Το δικαίωμα στο εκλέγειν και το εκλέγεσθαι είναι:,Ατομικό δικαίωμα,Πολιτικό δικαίωμα,Συλλογικό δικαίωμα,Κοινωνικό δικαίωμα
8,9,H ελεύθερη ανάπτυξη της προσωπικότητας:,Εμπεριέχει τη σεξουαλική ελευθερία,Δεν εξαρτάται από τα χρηστά ήθη,Είναι συλλογικό δικαίωμα,Δεν περιορίζεται
9,10,"Σύμφωνα με το άρθρο 120 του Συντάγματος, το δικαίωμα αντίστασης ασκείται σε περιπτώσεις:",Κατάλυσης του Συντάγματος,Αναθεώρησης του Συντάγματος,Αντισυνταγματικότητας ενός νόμου,Περιορισμού των ατομικών δικαιωμάτων


In [40]:
clean_df.to_csv('cleaned_questions_ASE.csv', index=False, encoding='utf-8-sig')

In [41]:
from io import StringIO

# Simulate the full content of the CSV cell as a multiline string
cell_content = """Στον παρακάτω πίνακα παρουσιάζονται τα κλιμάκια φορολογητέου εισοδήματος και
οι αντίστοιχοι οριακοί φορολογικοί συντελεστές.
Κλιμάκια Φορολογητέου Εισοδήματος (€),Οριακός Φορολογικός Συντελεστής (%)
0 – 10.000,10
10.001 – 30.000,20
30.001 και άνω,40
Η φορολογία στο εν λόγω παράδειγμα χαρακτηρίζεται ως:"""

# Split into lines
lines = cell_content.strip().split('\n')

# Find the header line (starts with "Κλιμάκια")
start_index = next(i for i, line in enumerate(lines) if line.startswith("Κλιμάκια"))
# Find the end of the table (next line after table that is not data)
end_index = next((i for i in range(start_index + 1, len(lines)) if ',' not in lines[i]), len(lines))

# Extract the table lines
table_lines = lines[start_index:end_index]

# Join them and read into a DataFrame
table_str = "\n".join(table_lines)
df = pd.read_csv(StringIO(table_str))

# Show the parsed table
df

,Κλιμάκια Φορολογητέου Εισοδήματος (€),Οριακός Φορολογικός Συντελεστής (%)
0,0 – 10.000,10
1,10.001 – 30.000,20
2,30.001 και άνω,40


In [8]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('cleaned_questions_ASEEEP.csv')

# Preview the data
df

,#,Question,a,b,c,d,correct_answer,category
0,1,Η ελευθερία έκφρασης γνώμης ασκείται:,Από τους/τις Έλληνες/ίδες πολίτες,Από όσους/ες ευρίσκονται στην Ελληνική επικράτεια,Από τους/τις Έλληνες/ίδες πολίτες και τους/τις Έλληνες/ίδες ομογενείς,Από τους/τις κατόχους της κοινής Ευρωπαϊκής ιθαγένειας,1,Συνταγματικό Δίκαιο
1,2,Τα βασανιστήρια:,Επιτρέπονται για την αποτροπή τρομοκρατικής επίθεσης,Δεν επιτρέπονται σε καμία περίπτωση,Επιτρέπονται με νόμο που ψηφίζει η Βουλή με τα 3/5 των μελών της,Επιτρέπονται για την προστασία της εδαφικής ακεραιότητας της χώρας,1,Συνταγματικό Δίκαιο
2,3,Το δικαίωμα επιλογής επαγγέλματος:,Αποτελεί έκφραση της ελευθερίας συμμετοχής στην κοινωνική ζωή της Χώρας,Δεν αποτελεί έκφραση της ελευθερίας συμμετοχής στην οικονομική ζωή της Χώρας,Δεν αποτελεί δικαίωμα,Είναι ειδική μορφή της οικονομικής ελευθερίας,3,Συνταγματικό Δίκαιο
3,4,Η ελευθερία ανάπτυξης της προσωπικότητας (άρ. 5 παρ. 1 Συντ.) ασκείται:,Μόνο από τους/τις Έλληνες/ίδες πολίτες,"Από τους/τις Έλληνες/ίδες πολίτες, τους/τις Ευρωπαίους/ες πολίτες, τους/τις αλλοδαπούς/ές και τους/τις ανιθαγενείς",Μόνο από τους/τις Έλληνες/ίδες και Ευρωπαίους/ες πολίτες,Μόνο από τους/τις Έλληνες/ίδες πολίτες και τους/τις ομογενείς,1,Συνταγματικό Δίκαιο
4,5,Κατοικία κατά την έννοια του άρ. 9 Συντ. αποτελεί:,Κάθε χώρος που ο άνθρωπος ορίζει ως χώρο διαβιώσεως και εργασίας,Κάθε χώρος που ο άνθρωπος ορίζει ως χώρο διαβιώσεως μόνο,Κάθε χώρος που ο άνθρωπος ορίζει ως χώρο εργασίας μόνο,Κάθε χώρος που ο άνθρωπος διαμένει έστω και για λίγες ώρες όπως το δωμάτιο ξενοδοχείου,3,Συνταγματικό Δίκαιο
5,6,Η ευθανασία:,Είναι συλλογικό δικαίωμα,Είναι κοινωνικό δικαίωμα,Δεν αναγνωρίζεται στο ελληνικό δίκαιο,Κατοχυρώνεται στο Σύνταγμα,2,Συνταγματικό Δίκαιο
6,7,Η αξία του ανθρώπου είναι:,Υποκειμενικό συνταγματικό δικαίωμα,Αντικειμενική συνταγματική αρχή,Νομοθετικός κανόνας,Πολιτικό δικαίωμα,1,Συνταγματικό Δίκαιο
7,8,Το δικαίωμα στο εκλέγειν και το εκλέγεσθαι είναι:,Ατομικό δικαίωμα,Πολιτικό δικαίωμα,Συλλογικό δικαίωμα,Κοινωνικό δικαίωμα,1,Συνταγματικό Δίκαιο
8,9,H ελεύθερη ανάπτυξη της προσωπικότητας:,Εμπεριέχει τη σεξουαλική ελευθερία,Δεν εξαρτάται από τα χρηστά ήθη,Είναι συλλογικό δικαίωμα,Δεν περιορίζεται,0,Συνταγματικό Δίκαιο
9,10,"Σύμφωνα με το άρθρο 120 του Συντάγματος, το δικαίωμα αντίστασης ασκείται σε περιπτώσεις:",Κατάλυσης του Συντάγματος,Αναθεώρησης του Συντάγματος,Αντισυνταγματικότητας ενός νόμου,Περιορισμού των ατομικών δικαιωμάτων,0,Συνταγματικό Δίκαιο


In [10]:
import pandas as pd
import csv

cols_to_quote = [1, 2, 3, 4, 5, 7]

# Convert those columns to strings (if not already)
for col in cols_to_quote:
    df.iloc[:, col] = df.iloc[:, col].astype(str)

# Custom function to selectively quote
def quote_selected_columns(val, col_index):
    if col_index in cols_to_quote:
        return f'"{val}"'
    return val

# Save manually with selective quoting
with open('output.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(df.columns)
    for _, row in df.iterrows():
        writer.writerow([
            quote_selected_columns(val, idx)
            for idx, val in enumerate(row)
        ])

In [11]:
df.to_csv('filename.csv', index=False)

In [12]:
import csv

# Replace with your actual DataFrame
data = df.values.tolist()

with open('custom_output.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL, quotechar='"')

    for row in data:
        formatted_row = [
            row[0],                           # 1st column (ID) – unquoted
            f'"{row[1]}"',                    # 2nd column – quoted
            f'"{row[2]}"',                    # 3rd column – quoted
            f'"{row[3]}"',                    # 4th column – quoted
            f'"{row[4]}"',                    # 5th column – quoted
            f'"{row[5]}"',                    # 6th column – quoted
            row[6],                           # 7th column – unquoted
            f'"{row[7]}"'                     # 8th column – quoted
        ]
        f.write(",".join(str(cell) for cell in formatted_row) + "\n")
